In [1]:
import os
import sys
import json

In [2]:
import opendatasets as od

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, col, expr, avg, count, sum, lit, when, isnan, isnull, udf, from_unixtime, StringType, month, year

In [2]:
!curl -L -o ./data/taxidemandfarepredictiondataset.zip https://www.kaggle.com/api/v1/datasets/download/sohaibanwaar1203/taxidemandfarepredictiondataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0 2848M    0 28.3M    0     0  25.0M      0  0:01:53  0:00:01  0:01:52 25.0M
  2 2848M    2 82.0M    0     0  37.7M      0  0:01:15  0:00:02  0:01:13 51.7M
  4 2848M    4  142M    0     0  45.3M      0  0:01:02  0:00:03  0:00:59 56.9M
  7 2848M    7  200M    0     0  48.5M      0  0:00:58  0:00:04  0:00:54 57.5M
  8 2848M    8  255M    0     0  49.7M      0  0:00:57  0:00:05  0:00:52 56.7M
 10 2848M   10  296M    0     0  48.2M      0  0:00:59  0:00:06  0:00:53 53.5M
 12 2848M   12  357M    0     0  50.1M      0  0:00:56  0:00:07  0:00:49 55.5M
 14 2848M   14  413M    0     0  50.7M      0  0:0

In [3]:
import zipfile

with zipfile.ZipFile('./data/taxidemandfarepredictiondataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [4]:
spark = SparkSession.builder \
    .appName("CargaMultiple") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "8g") \
    .getOrCreate()

In [5]:
# Donwload dataset
working_folder = os.getcwd()
dataset_folder = os.path.join(working_folder, 'data')
result_folder = os.path.join(working_folder, 'results')

In [6]:
# Carga todos los archivos CSV en un directorio
df201501 = spark.read.csv("./data/yellow_tripdata_2015-01.csv", header=True, inferSchema=True)

In [7]:
df201601 = spark.read.csv("./data/yellow_tripdata_2016-01.csv", header=True, inferSchema=True)

In [8]:
df201602 = spark.read.csv("./data/yellow_tripdata_2016-02.csv", header=True, inferSchema=True)

In [9]:
df201603 = spark.read.csv("./data/yellow_tripdata_2016-03.csv", header=True, inferSchema=True)

In [10]:
df = df201501.unionByName(df201601, allowMissingColumns=True) \
                    .unionByName(df201602, allowMissingColumns=True) \
                    .unionByName(df201603, allowMissingColumns=True)

In [11]:
# Limpiar dataset si pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude son 0 o nulos.
df = df.filter(col("pickup_longitude") != 0) \
       .filter(col("pickup_latitude") != 0) \
       .filter(col("dropoff_longitude") != 0) \
       .filter(col("dropoff_latitude") != 0)

In [12]:
# Crear columna Duración del Viaje
df = df.withColumn("duration_seconds", (unix_timestamp(col("tpep_dropoff_datetime"), "yyyy-MM-dd HH:mm:ss") -
                                        unix_timestamp(col("tpep_pickup_datetime"), "yyyy-MM-dd HH:mm:ss")).cast("int"))

In [13]:
# Crear columna Ingreso por milla, redondear 2 decimales
df = df.withColumn("income_per_mile", expr("round(total_amount / trip_distance, 2)"))

In [14]:
# Crear columna Ingreso por Kilometro, usando ingreso por milla y convirtiendo a kilometros
df = df.withColumn("income_per_km", expr("round(income_per_mile * 1.60934, 2)"))

In [15]:
# Agrupar por Mes con Agregate con Promedio de total_amount, Promedio de tip_amount, Promedio de trip_distance y Promedio de fare_amount
df = df.withColumn("month", month("tpep_dropoff_datetime"))
df = df.withColumn("year", year("tpep_dropoff_datetime"))
df_month_group = df.groupBy("month", "year").agg(avg("total_amount"), avg("tip_amount"), avg("trip_distance"), avg("fare_amount"))

In [16]:
# Agrupar por Zona geografica con Agregate con Promedio de total_amount, Promedio de tip_amount, Promedio de trip_distance y Promedio de fare_amount
df_geo_group = df.groupBy("dropoff_longitude", "dropoff_latitude").agg(
    avg("total_amount").alias("avg_total_amount"),
    avg("tip_amount").alias("avg_tip_amount"),
    avg("trip_distance").alias("avg_trip_distance"),
    avg("fare_amount").alias("avg_fare_amount"),
    count("*").alias("trip_count")  # Count the number of trips in each location
)

In [17]:
parquet_path = os.path.join(result_folder, "taxi_data.parquet")
df.write.partitionBy("year", "month").option("compression", "snappy").parquet(parquet_path, mode="overwrite")

Py4JJavaError: An error occurred while calling o133.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j